# 🧬 Geometry-Complete Equivariant Diffusion
## De Novo Drug Design Training

**Data**: Local Colab disk (fast)
**Code/Checkpoints**: Google Drive (persistent)

## Cell 1: Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import torch
print(f'GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None!"}')

!pip install -q torch-geometric rdkit scipy numpy pyyaml tqdm gdown scikit-learn
print('✅ Setup complete')

## Cell 2: Clone Repo (Drive for persistence)

In [ ]:
import os
REPO = '/content/drive/MyDrive/geom_diffusion'
if not os.path.exists(REPO):
    !git clone https://github.com/Nethrananda21/geom_diffusion.git {REPO}
%cd {REPO}
!git pull origin master

## Cell 3: Download Data to LOCAL disk (fast!)

In [ ]:
import os
import gdown

# LOCAL disk - fast I/O!
DATA_DIR = '/content/data/crossdocked'
os.makedirs(DATA_DIR, exist_ok=True)

files = {
    'train_data.pkl': '1vJyxCIqCYwP3qj4THMofdSd1rZDEQpPG',
    'val_data.pkl': '1FpVNcdj0R5YOsaLQm6T4D5QOKZGI4Xc5'
}

for fname, fid in files.items():
    path = f'{DATA_DIR}/{fname}'
    if not os.path.exists(path):
        print(f'📥 Downloading {fname} to LOCAL disk...')
        gdown.download(id=fid, output=path, quiet=False)
    else:
        print(f'✅ {fname} exists')

print(f'\n📁 Data location: {DATA_DIR} (LOCAL - fast!)')

## Cell 4: Create 5K Subset (LOCAL processing)

In [ ]:
import pickle
import numpy as np
from collections import defaultdict
from sklearn.cluster import KMeans

np.random.seed(42)

# Load from LOCAL
with open(f'{DATA_DIR}/train_data.pkl', 'rb') as f:
    full_data = pickle.load(f)
print(f'Loaded {len(full_data)} samples')

# Group by pocket
pockets = defaultdict(list)
pocket_info = {}

for i, sample in enumerate(full_data):
    if hasattr(sample, 'keys'):
        pocket_id = sample.get('pocket_id', sample.get('receptor', f'pocket_{i}'))
        pocket_coords = np.array(sample.get('pocket_coords', sample.get('pocket_pos', [])))
    else:
        pocket_id = getattr(sample, 'pocket_id', f'pocket_{i}')
        pocket_coords = sample.pocket_pos.numpy() if hasattr(sample, 'pocket_pos') else np.zeros((1,3))
    
    pockets[pocket_id].append(sample)
    if pocket_id not in pocket_info:
        pocket_info[pocket_id] = {
            'size': len(pocket_coords),
            'centroid': pocket_coords.mean(axis=0) if len(pocket_coords) > 0 else np.zeros(3)
        }

print(f'Found {len(pockets)} unique pockets')

# Filter
MAX_SIZE, MIN_LIGANDS = 250, 50
valid = [p for p, s in pockets.items() 
         if pocket_info[p]['size'] <= MAX_SIZE and len(s) >= MIN_LIGANDS]
print(f'After filter: {len(valid)} pockets')

# Stratify
small = [p for p in valid if pocket_info[p]['size'] <= 150]
medium = [p for p in valid if 150 < pocket_info[p]['size'] <= 200]
large = [p for p in valid if 200 < pocket_info[p]['size'] <= 250]

print(f'Bins: Small={len(small)}, Medium={len(medium)}, Large={len(large)}')

# K-means diversity
def select_diverse(plist, n):
    if len(plist) <= n: return plist
    centroids = np.array([pocket_info[p]['centroid'] for p in plist])
    km = KMeans(n_clusters=n, random_state=42, n_init=10).fit(centroids)
    return [plist[[i for i in range(len(plist)) if km.labels_[i] == c][0]] for c in range(n) if any(km.labels_ == c)]

sel_small = select_diverse(small, min(40, len(small)))
sel_medium = select_diverse(medium, min(40, len(medium)))
sel_large = select_diverse(large, min(40, len(large)))

# Stratified split
np.random.shuffle(sel_small); np.random.shuffle(sel_medium); np.random.shuffle(sel_large)

train_pockets = sel_small[:33] + sel_medium[:33] + sel_large[:34]
val_pockets = sel_small[33:40] + sel_medium[33:40] + sel_large[34:40]

print(f'\n✅ Train: {len(train_pockets)} pockets, Val: {len(val_pockets)} pockets')
print(f'✅ Overlap: {len(set(train_pockets) & set(val_pockets))}')

# Create datasets
train_samples = [s for p in train_pockets for s in pockets[p][:50]]
val_samples = [s for p in val_pockets for s in pockets[p][:50]]

print(f'📊 Train: {len(train_samples)}, Val: {len(val_samples)}')

# Save to LOCAL
with open(f'{DATA_DIR}/train_5k.pkl', 'wb') as f:
    pickle.dump(train_samples, f)
with open(f'{DATA_DIR}/val_1k.pkl', 'wb') as f:
    pickle.dump(val_samples, f)

print('💾 Saved to LOCAL disk')

## Cell 5: Update Config (point to LOCAL data)

In [ ]:
import yaml

%cd /content/drive/MyDrive/geom_diffusion

with open('configs/debug_t4.yaml', 'r') as f:
    cfg = yaml.safe_load(f)

# Point to LOCAL data (fast!)
cfg['data']['root'] = '/content/data'
cfg['data']['train_file'] = 'crossdocked/train_5k.pkl'
cfg['data']['val_file'] = 'crossdocked/val_1k.pkl'
cfg['training']['max_epochs'] = 50
cfg['training']['batch_size'] = 4
cfg['hardware']['num_workers'] = 2

with open('configs/debug_t4.yaml', 'w') as f:
    yaml.dump(cfg, f)

print('✅ Config: data=/content/data (LOCAL), checkpoints=Drive')

## Cell 6: Delete Cache

In [ ]:
import shutil
from pathlib import Path

for cache in ['/content/data/cache', 'data/cache']:
    if Path(cache).exists():
        shutil.rmtree(cache)
        print(f'🗑️ Deleted {cache}')

## Cell 7: Train 🚀 (Checkpoints save to Drive)

In [ ]:
%cd /content/drive/MyDrive/geom_diffusion

# Checkpoints go to Drive (persistent)
!python train.py --config configs/debug_t4.yaml --checkpoint_dir checkpoints

## Cell 8: Resume (After Disconnect)

In [ ]:
# After disconnect:
# 1. Run Cells 1, 2, 3, 4, 5, 6 (re-download data to LOCAL)
# 2. Then run this cell:

# %cd /content/drive/MyDrive/geom_diffusion
# !python train.py --config configs/debug_t4.yaml --resume checkpoints/best_model.pt